# Generating Images from Models

In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from src.models import unet_generator
from src.generate_image import generate_image, GAN_image_folder
import os, shutil
from sys import argv
from src.make_dataset import parse_and_save_dir

First, we need to load the parameters that we used in the cyclegan training:

`identifier`: This will be the name of the model that will be useful for saving and loading checkpoints

`parent_dir`: Location of the `cycle_gan` repository

`sim_dir_raw`: The path to the folder containing simulated images

`gaussian`: Set this parameter to some number if you want to add gaussian noise with a given stddev to the simulation data

`num_channels`: The number of channels of the experimental images

`chkpt_range`: The range of checkpoint indices we want to use to generate the images. If `None`, generates a folder for every checkpoint

`sim_dir_raw`: The path to the folder of simulated images we want to GAN


Generated images will be saved in 

`ganned_dir = parent_dir + "data/GAN/exp/{}_CKPT_{}/".format(identifier, chkpt_idx)`


In [ ]:
identifier = "20240426_DATA_brightfield_mixed_edit"

parent_dir = "/mnt/d/cycle_gan/"

gaussian   = 0.1 

num_channels=1

chkpt_range = -1

sim_dir_raw = parent_dir + "data/sim/brightfield_mixed_synth/"

Next, we load the checkpoint

In [ ]:
checkpoint_path = parent_dir + "cycle_gan_results/checkpoints/checkpoint_{}".format(identifier)

num_channels = 1
generator_exp = unet_generator(num_channels, 1, "instancenorm")
generator_sim = unet_generator(1, num_channels, "instancenorm")

# ## Checkpoints
checkpoint_path = parent_dir + "cycle_gan_results/checkpoints/checkpoint_{}".format(identifier)

ckpt = tf.train.Checkpoint(generator_exp=generator_exp,
                           generator_sim=generator_sim)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=None)

# if a checkpoint exists, restore the latest checkpoint.
num_ckpts = len(ckpt_manager.checkpoints)
print(" number of checkpoints ", num_ckpts)
if chkpt_range == None:
    chkpt_range = range(num_ckpts)
if isinstance(chkpt_range, int):
    chkpt_range = [chkpt_range]

def cycle(x):
    return generator_sim(generator_exp(x))

Next, we parse the simulated images

In [4]:
sim_dir = sim_dir_raw[:-1] + "_gauss_{}_256_slices_TO_GAN/".format(gaussian)
parse_and_save_dir(sim_dir_raw, sim_dir, 256, 256, num_channels=num_channels, gaussian=gaussian)

Finally, we gann the folder containing the images

In [ ]:
#GAN A FOLDER
for i in chkpt_range:
    ckpt.restore(ckpt_manager.checkpoints[i])
    print("Restored from {}".format(ckpt_manager.checkpoints[i]))
    print("Cycle GAN")

    model      = generator_exp
    data_dir   = sim_dir
    ganned_dir = parent_dir + "data/GAN/exp/{}_CKPT_{}/".format(identifier, i)
    GAN_image_folder(model, data_dir, ganned_dir, 256, 256, False)